**Cylindrical Thermal Convection**

- Compared to *Cylindrical scaling for dynamical cooling models of the Earth* (van Keken, 2001).
- van Keken measured diagnostic model outputs (velocities and heat-flow), which we can make comparisons to.
- The cylindrical model of van Keken is reproduced by using inner and outer radii of 0.4292 and 1.4292 respectively. Comparisons are made for $Ra = 10^5$.

**Limitations:** The models of van Keken have free-slip walls, whereas the following are no-slip. They also have a shear-heating term. However, approximate comparisons of the diagnostic quantities is still useful.


In [ ]:
import underworld as uw
import glucifer
import numpy as np
from underworld import function as fn
uw.matplotlib_inline()
import math

In [ ]:
import matplotlib.pyplot as plt
plt.ion()

In [ ]:
# Whether or not to load the existing temperature field
loaddata = True

In [ ]:
annulus = uw.mesh._FeMesh_Annulus(elementRes=(32,64), 
                                  radialLengths=(0.4292,1.4292), angularExtent=(0.,360.),
                                  periodic = [False, True])


tField = uw.mesh.MeshVariable(annulus, nodeDofCount=1)
vField = uw.mesh.MeshVariable(annulus, nodeDofCount=2)
pField = uw.mesh.MeshVariable(annulus.subMesh, nodeDofCount=1)

In [ ]:
Ra = 1e5

In [ ]:
fig = glucifer.Figure()
fig.append(glucifer.objects.Mesh(annulus, segmentsPerEdge=1))
fig.append(glucifer.objects.Surface(annulus, tField, onMesh=True ))
fig.show()

In [ ]:
jWalls = annulus.specialSets["MinJ_VertexSet"] + annulus.specialSets["MaxJ_VertexSet"]

In [ ]:
t_outer = 0.0
t_inner = 1.0

outer = annulus.specialSets["MaxI_VertexSet"]
inner = annulus.specialSets["MinI_VertexSet"]


In [ ]:

# setup parameters for temperature distribution
dr = annulus.radialLengths[1] - annulus.radialLengths[0]
dT_dr = (t_outer-t_inner)/(dr)
c0 = t_inner - dT_dr*annulus.radialLengths[0]

# wavenumber for perturbation
k = 3.5

for ind,coord in enumerate(annulus.data):
    r = np.sqrt(coord[0]**2 + coord[1]**2)
    theta = np.arctan2(coord[1], coord[0])
    
    pert = 0.2 *np.sin(k*theta)
    
    t = r*dT_dr + c0
    tField.data[ind] = min([max([0.,t + 1.*pert]),1])
    
tField.data[inner.data] = t_inner
tField.data[outer.data] = t_outer
    
if loaddata:
    tField.load('vanKeken2001data/temp.h5',interpolate=True)

In [ ]:
# A hacky  way to import data from a different resolution, as the loaddata interpolation
# doesn't like the inner and outer surfaces of the cylindrical geometry

importlowres = False

if importlowres:
    tempannulus = uw.mesh._FeMesh_Annulus(elementRes=(16,32), 
                                      radialLengths=(0.4292,1.4292), angularExtent=(0.,360.),
                                      periodic = [False, True])
    tempTfield = uw.mesh.MeshVariable(tempannulus, nodeDofCount=1)
    tempTfield.load('temp.h5')

    for node in range(annulus.nodesGlobal):
        pos = annulus.data[node]
        if node not in inner and node not in outer:
            tField.data[node] = tempTfield.evaluate(tuple(pos))


In [ ]:
fig.show()

In [ ]:
vBC = uw.conditions.DirichletCondition( variable=vField, indexSetsPerDof=(inner+outer, inner+outer))
tBC = uw.conditions.DirichletCondition( variable=tField, indexSetsPerDof=(inner+outer))

In [ ]:
tDotField = uw.mesh.MeshVariable(annulus, nodeDofCount=1)
advDiffSLE = uw.systems.AdvectionDiffusion(tField, tDotField, vField, fn_diffusivity=1.0, conditions=tBC)

In [ ]:
g  = 1.0*annulus.fn_unitvec_radial()

In [ ]:
bodyForceFn = g * tField * Ra / (annulus.radialLengths[1]-annulus.radialLengths[0])

In [ ]:
stokesSLE = uw.systems.Stokes( vField, pField, fn_viscosity=1.0, fn_bodyforce=bodyForceFn, conditions=vBC)

In [ ]:
stokesSolver = uw.systems.Solver(stokesSLE)

In [ ]:
nS = 10
arrT = np.zeros(nS)
for i in range(nS):
    stokesSolver.solve()


    dt = advDiffSLE.get_max_dt()
    advDiffSLE.integrate(dt)
    
    arrT[i] = np.average(tField.data[:])

plt.plot(np.arange(nS),arrT)
plt.ylim([0.25,0.5])
plt.savefig('avtemp.pdf')


In [ ]:
figV = glucifer.Figure(**fig)
figV.append(glucifer.objects.Surface(annulus, tField,onMesh=True))

figV.append(glucifer.objects.VectorArrows(annulus, vField / Ra  * 100. , onMesh=True,arrowHead = 0.3))

figV.show()
figV.save_image('temperature.png')

In [ ]:
# Save data
mH = annulus.save("vanKeken2001data/mesh.h5")
tF = tField.save('vanKeken2001data/temp.h5',mH)


**Compare quantities from van Keken 2001**

In [ ]:
# Compared to van Keken 114.4
vdotv = fn.math.dot(vField,vField)
v2sum_integral  = uw.utils.Integral( mesh=annulus, fn=vdotv )
volume_integral = uw.utils.Integral( mesh=annulus, fn=1. )
tgrad = fn.math.dot(tField.fn_gradient,annulus.fn_unitvec_radial())


In [ ]:
vrms = np.sqrt(v2sum_integral.evaluate()) / volume_integral.evaluate()
print("for Ra=1e5, van Keken measured a RMS velocity of 114.4")
print("Our v_rms is %.2f" %vrms)

In [ ]:

vmax = np.max(vField.data[:])
print("for Ra=1e5, van Keken measured a maximum velocity of 178.")
print("Our v_max is %.2f" %vmax)


In [ ]:
tSurf_integral  = uw.utils.Integral( mesh=annulus, fn=tgrad, integrationType="surface", surfaceIndexSet=outer )
tSurf_av = -1. * tSurf_integral.evaluate()[0] / (2.*np.pi *annulus.radialLengths[1])

print("for Ra=1e5, van Keken measured a inner average heat-flow of 3.72")
print("Our inner heat-flow is %.2f" %tSurf_av)




In [ ]:
tSurf_integral  = uw.utils.Integral( mesh=annulus, fn=tgrad, integrationType="surface", surfaceIndexSet=inner )
tSurf_av = -1. * tSurf_integral.evaluate()[0] / (2.*np.pi *annulus.radialLengths[0])

print("for Ra=1e5, van Keken measured an outer average heat-flow of 12.42")
print("Our outer heat-flow is %.2f" %tSurf_av)

The largest inconsistency between the models is the $v_{rms}$, which is likely to be the result of our use of fixed inner and outer velocity BCs. The other measurements are close, especially given the difference in BC and the absence of shear heating in our models.